In [75]:
from sklearn import datasets,metrics
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')
type(mnist)


sklearn.utils._bunch.Bunch

In [76]:
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [77]:
mnist.frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 785 entries, pixel1 to class
dtypes: category(1), int64(784)
memory usage: 418.8 MB


In [78]:
pd.DataFrame(mnist.data).head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
data_df = pd.read_csv('digit-recognizer\\train.csv')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [90]:
import numpy as np
n = len(data_df)
one_third =  n // 3

data_df_1 = data_df.iloc[:one_third]
data_df_2 = data_df.iloc[one_third:2*one_third]
data_df_3 = data_df.iloc[2*one_third:]

def split_df(data_df) :
    return data_df['label'].to_numpy(), data_df.drop(columns=['label']).to_numpy()


(14000, 784)

In [ ]:
y_1 , X_train_1 = split_df(data_df_1)
y_2 , X_train_2 = split_df(data_df_2)
y_3 , X_train_3 = split_df(data_df_3)

In [105]:
from skimage.transform import AffineTransform, warp

def shift(image, vector):
    transform = AffineTransform(translation=vector)
    return warp(image, transform, mode='warp', preserve_range=False)

In [153]:
y, X = split_df(data_df)
X_2D = X.reshape((len(X),28,28))

X_shifted = shift(X_2D,(2,0))
y_shift = y

X_shifted = X_shifted.reshape(len(X_shifted),28*28)
X_shifted.shape

(42000, 784)

In [172]:
def cat(x_train,x_shifted):
    x_cat = np.concatenate((x_train,x_shifted))
    return x_cat

train_fold_1 = (cat(X_train_1, X_shifted),cat(y_1, y_shift))
train_fold_2 = (cat(X_train_2, X_shifted),cat(y_2, y_shift))
train_fold_3 = (cat(X_train_3, X_shifted),cat(y_3, y_shift))
train_fold = [train_fold_1, train_fold_2, train_fold_3]

In [174]:
val_fold_1 = (cat(X_train_2,X_train_3), cat(y_2,y_3))
val_fold_2 = (cat(X_train_1,X_train_3), cat(y_1,y_3))
val_fold_3 = (cat(X_train_1,X_train_2), cat(y_1,y_2))
val_fold = [val_fold_1, val_fold_2, val_fold_3]

In [183]:
from sklearn.linear_model import SGDClassifier
sgd_clf = SGDClassifier()

def one_training(sgd_clf, x_train, y_train, x_val, y_val, classes=None):
    if classes is not None:
        sgd_clf.partial_fit(x_train, y_train, classes=classes)
    else:
        sgd_clf.partial_fit(x_train, y_train)
    score = sgd_clf.score(x_val, y_val)
    return score

scores = []
for i, (train_data, val_data) in enumerate(zip(train_fold, val_fold)):
    x_train, y_train = train_data
    x_val, y_val = val_data
    score = one_training(sgd_clf, x_train, y_train, x_val, y_val, classes=np.unique(y))
    scores.append(score)
    print(f"Fold {i+1} score: {score}")

average_score = sum(scores) / len(scores)
print(f"Average score across folds: {average_score}")

Fold 1 score: 0.8326428571428571
Fold 2 score: 0.85675
Fold 3 score: 0.8687142857142857
Average score across folds: 0.8527023809523809


In [203]:
test_df = pd.read_csv('digit-recognizer\\test.csv')
X_test = test_df.to_numpy()
pred = sgd_clf.predict(X_test)
predictions_df = pd.DataFrame({
    'ImageId': range(1, len(pred) + 1),
    'Label': pred
})

# Save the DataFrame to a CSV file
predictions_df.to_csv('predictions.csv', index=False)

(28000,)